In [1]:
from dotenv import load_dotenv

from frames.data.figstep import QueryType, SafeBenchLanguages
from frames.experiments import MultilingualModelGenerator
from frames.utils.settings import load_models

load_dotenv()

MODELS = load_models().to_dict(orient="index").values()
QUERY_TYPES = [QueryType.baseline]
GUIDE = ["safe.n.01", "harm.v.01"]
DB_PATH = "results.shelf"

generator = MultilingualModelGenerator(
    db_path=DB_PATH,
    models=MODELS,
    query_types=QUERY_TYPES,
    languages=SafeBenchLanguages,
    guide=GUIDE,
)

In [ ]:
generator.generate_all()

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

2025-02-05 16:11:31.031 | INFO     | frames.models.hf.base:__init__:88 - Loaded model: meta-llama/Llama-3.1-8B-Instruct
2025-02-05 16:11:31.032 | WARNING  | frames.models.hf.base:__init__:89 - memory cost: 15316 Mb
 19%|█▉        | 3/16 [02:58<12:50, 59.26s/it]